In [1]:
import os
import numpy as np
from tqdm import tqdm
import albumentations as A

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers, models
from keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.model_selection import train_test_split

/opt/homebrew/anaconda3/envs/tensorflow/lib/python3.8/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
# Define augmentation pipeline
augmentation_pipeline = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.5),
    A.GaussianBlur(var_limit=(10.0, 50.0), p=0.2),
    A.RandomGamma(gamma_limit=(80, 120), p=0.2),
    A.RandomResizedCrop(height=256, width=256, scale=(0.8, 1.0), p=0.3)
])

# Augment images and masks
def augment_image_and_mask(image, mask):
    # pass image and mask to augmentation pipeline so that the same transformation is applied to both
    augmented = augmentation_pipeline(image=image, mask=mask)
    return augmented['image'], augmented['mask']

/var/folders/fl/ngxwxtkd09l10m23llp7drg80000gn/T/ipykernel_65438/2635536139.py:7: UserWarning: Argument 'alpha_affine' is not valid and will be ignored.
  A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.5),
/var/folders/fl/ngxwxtkd09l10m23llp7drg80000gn/T/ipykernel_65438/2635536139.py:8: UserWarning: Argument 'var_limit' is not valid and will be ignored.
  A.GaussianBlur(var_limit=(10.0, 50.0), p=0.2),


In [3]:
# Main function to load data from train and test directories
def load_dataset_refuge(train_dir, val_dir, test_dir, image_size):
    all_images = []
    all_masks = []
    
    train_img_dir = os.path.join(train_dir, 'Images')
    train_mask_dir = os.path.join(train_dir, 'gts')
    
    val_img_dir = os.path.join(val_dir, 'Images')
    val_mask_dir = os.path.join(val_dir, 'gts')
    
    test_img_dir = os.path.join(test_dir, 'Images')
    test_mask_dir = os.path.join(test_dir, 'gts')
    
    for img_path in tqdm(os.listdir(train_img_dir)):
        basename = img_path.split('.')[0]
        img_path = os.path.join(train_img_dir, img_path)
        mask_path = os.path.join(train_mask_dir, basename + '.bmp')
        
        img = load_img(img_path, target_size=image_size)
        mask = load_img(mask_path, target_size=image_size, color_mode="grayscale")
        img_array = img_to_array(img) / 255.0
        mask_array = img_to_array(mask) / 255.0
        
        img_array, mask_array = augment_image_and_mask(img_array, mask_array)
    
        all_images.append(img_array)
        all_masks.append(mask_array)

    for img_path in tqdm(os.listdir(val_img_dir)):
        basename = img_path.split('.')[0]
        img_path = os.path.join(val_img_dir, img_path)
        mask_path = os.path.join(val_mask_dir, basename + '.bmp')
        
        img = load_img(img_path, target_size=image_size)
        mask = load_img(mask_path, target_size=image_size, color_mode="grayscale")
        img_array = img_to_array(img) / 255.0
        mask_array = img_to_array(mask) / 255.0
        
        img_array, mask_array = augment_image_and_mask(img_array, mask_array)
        
        all_images.append(img_array)
        all_masks.append(mask_array)
    
    for img_path in tqdm(os.listdir(test_img_dir)):
        basename = img_path.split('.')[0]
        img_path = os.path.join(test_img_dir, img_path)
        mask_path = os.path.join(test_mask_dir, basename + '.bmp')
        
        img = load_img(img_path, target_size=image_size)
        mask = load_img(mask_path, target_size=image_size, color_mode="grayscale")
        img_array = img_to_array(img) / 255.0
        mask_array = img_to_array(mask) / 255.0
        
        img_array, mask_array = augment_image_and_mask(img_array, mask_array)
    
        all_images.append(img_array)
        all_masks.append(mask_array)
        
    return np.array(all_images), np.array(all_masks)

In [4]:
# Directories for images and masks
train_dir = 'path/to/REFUGE/train'
val_dir = 'path/to/REFUGE/val'
test_dir = 'path/to/REFUGE/test'
image_size = (256, 256)

all_images, all_masks = load_dataset_refuge(train_dir, val_dir, test_dir, image_size)

# Split into training and test sets (80% train, 20% test)
train_images, test_images, train_masks, test_masks = train_test_split(
    all_images, all_masks, test_size=0.3, random_state=42
)

# split train set into training and validation sets (90% train, 10% validation)
val_images, test_images, val_masks, test_masks = train_test_split(
    test_images, test_masks, test_size=0.5, random_state=42
)

100%|██████████| 400/400 [00:05<00:00, 74.38it/s]


In [5]:
train_images.shape, val_images.shape, test_images.shape

((840, 256, 256, 3), (180, 256, 256, 3), (180, 256, 256, 3))

In [6]:
train_masks.shape, val_masks.shape, test_masks.shape

((840, 256, 256, 1), (180, 256, 256, 1), (180, 256, 256, 1))

In [7]:
def unet_model(input_size=(256, 256, 3)):
    inputs = layers.Input(input_size)
    
    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Bottleneck
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    
    # Decoder
    up1 = layers.UpSampling2D(size=(2, 2))(conv3)
    up1 = layers.concatenate([up1, conv2], axis=-1)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(up1)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv4)
    
    up2 = layers.UpSampling2D(size=(2, 2))(conv4)
    up2 = layers.concatenate([up2, conv1], axis=-1)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(up2)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv5)
    
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv5)  # Sigmoid for binary classification
    
    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

In [8]:
model = unet_model(input_size=(256, 256, 3))
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam',
              loss='binary_crossentropy', 
              metrics=['accuracy'])

Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2024-12-06 11:33:06.817949: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-06 11:33:06.818582: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 64  36928       ['conv2d[0][0]']                 
                                )                                                             

In [10]:
# Train the model
early_stopping = EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

history = model.fit(train_images, train_masks, epochs=50,
                    batch_size=16, verbose=1, steps_per_epoch=len(train_images) // 16,
                    validation_data=(val_images, val_masks),
                    callbacks=[early_stopping])

Epoch 1/50


2024-12-06 11:33:09.397691: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-12-06 11:33:09.747182: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


52/52 [==============================] - ETA: 0s - loss: 0.1108 - accuracy: 0.9785

2024-12-06 11:34:25.299461: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


52/52 [==============================] - 81s 2s/step - loss: 0.1108 - accuracy: 0.9785 - val_loss: 0.0351 - val_accuracy: 0.9831
Epoch 2/50
52/52 [==============================] - 69s 1s/step - loss: 0.0327 - accuracy: 0.9827 - val_loss: 0.0266 - val_accuracy: 0.9831
Epoch 3/50
52/52 [==============================] - 67s 1s/step - loss: 0.0260 - accuracy: 0.9828 - val_loss: 0.0192 - val_accuracy: 0.9831
Epoch 4/50
52/52 [==============================] - 71s 1s/step - loss: 0.0211 - accuracy: 0.9845 - val_loss: 0.0186 - val_accuracy: 0.9857
Epoch 5/50
52/52 [==============================] - 63s 1s/step - loss: 0.0174 - accuracy: 0.9858 - val_loss: 0.0152 - val_accuracy: 0.9867
Epoch 6/50
52/52 [==============================] - 68s 1s/step - loss: 0.0174 - accuracy: 0.9857 - val_loss: 0.0164 - val_accuracy: 0.9864
Epoch 7/50
52/52 [==============================] - 69s 1s/step - loss: 0.0157 - accuracy: 0.9862 - val_loss: 0.0155 - val_accuracy: 0.9867
Epoch 8/50
52/52 [=============

In [11]:
scores = model.evaluate(val_images, val_masks)
print("Validation Loss: %.2f" % scores[0])
print("Validation Accuracy: %.2f%%" % (scores[1] * 100))

6/6 [==============================] - 6s 871ms/step - loss: 0.0117 - accuracy: 0.9875
Validation Loss: 0.01
Validation Accuracy: 98.75%


In [12]:
scores = model.evaluate(test_images, test_masks)
print("Test Loss: %.2f" % scores[0])
print("Test Accuracy: %.2f%%" % (scores[1] * 100))

6/6 [==============================] - 5s 793ms/step - loss: 0.0131 - accuracy: 0.9870
Test Loss: 0.01
Test Accuracy: 98.70%


In [13]:
# Save the model
model_json = model.to_json()
with open('../models_segmentation/ODOC.json', 'w') as json_file:
    json_file.write(model_json)

# Saving the model and weights
model.save_weights('../models_segmentation/ODOC.weights.h5')